# Rolling horizon forecast

In [ ]:
import pandas as pd
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from bokeh.io import output_file, show
from bokeh.layouts import column
from bokeh.plotting import figure

# Load some data
df = pd.read_pickle('data/pickles/df_aa-offshore_res1h.pickle')
# df = pd.read_pickle('data/pickles/df_ps_pv_res1h.pickle')
resolution_ = '1H'
preconditions = dict(yearly_seasonality = True, daily_seasonality = True)

# Prepare data and model for fbprophet
df = df.reset_index()                                       # Move the datetime index column as a separate data column 
df = df.rename(columns={'datetime':'ds', 'actual':'y'})     # Rename the datetime and data column for use in fbprophet
model = Prophet(**preconditions)                            # Precondition the model to look for certain trends and seasonalities
model.fit(df)

# Select some window and resolution for the forecast
start_ = model.history_dates.min()
end_ = model.history_dates.max()

dates = pd.date_range(start = start_, end = end_, freq = resolution_)
window = pd.DataFrame({'ds': dates})

# Cheap rolling horizon forecast
forecast = model.predict(window)
df.set_index('ds', inplace=True)
forecast.set_index('ds', inplace=True)
viz_df = df.join(forecast[['yhat', 'yhat_lower','yhat_upper']], how = 'outer')

# # Expensive but proper rolling horizon forecast
# initial_ = start_ - model.history_dates.min()               # Training period
# horizon_ = resolution_                                      # How far we look ahead (should be equal to the resolution)
# period_ = resolution_                                       # Period before we repeat the model (should be equal to the resolution)
# df_cv = cross_validation(model, horizon = horizon_, initial = initial_, period = period_)


In [ ]:
# Plotting
x = viz_df.index
y1 = viz_df.y
y2 = viz_df.yhat

s1 = figure(plot_width=1400, plot_height=750, title=None)
s1.circle(x, y1, size=4, color="navy", alpha=0.5)
s1.square(x, y2, size=4, color="orange", alpha=0.5)

show(s1)